In [1]:
#Required libraries
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome

In [2]:
# A function to extract tweet data from a tweet

def get_tweet_data(card):
    
    username = card.find_element_by_xpath(".//span").text
    try:
        twitterhandle = card.find_element_by_xpath(".//span[contains(text(), '@')]").text 
    except NoSuchElementException:
        return
    try:
        postdate = card.find_element_by_xpath(".//time").get_attribute("datetime")
    except NoSuchElementException:
        return

    responding = card.find_element_by_xpath(".//div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]").text
    tweet = card.find_element_by_xpath(".//div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]").text
    

    tweet = tweet if responding.startswith("Replying") else responding + tweet
    
    comment_count = card.find_element_by_xpath(".//div[@data-testid='reply']").text
    retweet_count = card.find_element_by_xpath(".//div[@data-testid='retweet']").text
    like_count = card.find_element_by_xpath(".//div[@data-testid='like']").text
    
    tweet_data = (username, twitterhandle, postdate, tweet, comment_count, retweet_count, like_count)
    return tweet_data

In [3]:
# Create instance of webdriver and navigate to login screen
driver = Chrome(r"__filepath to chrome weddriver__")
driver.get("https://twitter.com/login")
driver.maximize_window()

# Enter username and password
username = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//input[@autocomplete='username']")))
username.clear()
username.send_keys("__username/email__")
Next = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Next')]"))).click()

# Login into Twitter using password
password = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//input[@name='password']")))
password.clear()
my_password = "__Password__"
password.send_keys(my_password)
login = driver.find_element_by_xpath("//span[contains(text(), 'Log in')]").click()

#Enter search term
keyword = "#polynote"
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@aria-label='Search query']")))
searchbox.send_keys(keyword)
searchbox.send_keys(Keys.RETURN)
WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Latest')]"))).click()
sleep(5)

C:\Users\TOFUNMI\anaconda3\envs\Web Scraping\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  
C:\Users\TOFUNMI\anaconda3\envs\Web Scraping\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [4]:
#Extract all tweets on the page
tweets_data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True
                
while scrolling:
    page_cards = driver.find_elements_by_xpath("//article[@data-testid='tweet']")
    for card in page_cards[-15:]:
        data = get_tweet_data(card)
        if data:
            tweet_id = ''.join(data)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                tweets_data.append(data)

    scroll_attempt = 0
    while True:
        #scroll and wait for 2 seconds
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        current_position = driver.execute_script("return window.pageYOffset;")
        if last_position == current_position:
            scroll_attempt += 1
            
            #attempt to scroll 3 times and if scroll position does not change set the outer while loop to false causing it to
            #break and break out of inner while loop
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = current_position
            break

# close the web driver
driver.close()

C:\Users\TOFUNMI\anaconda3\envs\Web Scraping\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  
C:\Users\TOFUNMI\anaconda3\envs\Web Scraping\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [5]:
# Save data to a csv file
with open(keyword + " tweets.csv", "w", newline="", encoding="utf-8") as f:
    header = ["Username", "Twitter Handle", "Timestamp", "Text", "Comments", "Retweets", "Likes"]
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(tweets_data)